## 재무제표 db에 넣기

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [7]:
# 삼성전자 네이버증권 페이지
url = 'https://finance.naver.com/item/coinfo.naver?code=005930'

In [8]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [9]:
# 매출액 행의 매출액 셀
path= '#RVArcVR1a2 > table:nth-child(2) > tbody > tr > th'


In [10]:
tds = soup.select('table > tbody > tr > th')
tds

[<th scope="row">
 	 								PER<span class="bar">l</span>EPS<span class="date">(2021.09)</span><a href="javascript:togglePannel('helpPannel1');" onmouseout="hidePannel('helpPannel1');" onmouseover="showPannel('helpPannel1');"><img alt="추정PER EPS 상세설명" height="11" src="https://ssl.pstatic.net/static/nfinance/2018/10/26/btn_help.png" width="11"/></a>
 <div class="lyr_section">
 <div class="tooltip_lyr" id="helpPannel1" style="display:none">
 <p>
 <strong>PER = 현재가 ÷ EPS</strong>
 												EPS는 지배기업귀속 최근 4분기 합산 순이익을
 												수정평균발행주식수로 나눈 값이며, 보통주와
 												우선주를 합산해서 계산합니다.
 											</p>
 <span class="arrow"></span>
 </div>
 </div>
 </th>]

In [11]:
# requests 로는 불가능 판단

In [12]:
# 셀레늄 사용

# driver = webdriver.Chrome()
# driver.get(url)
# driver.find_element_by_id('cns_Tab22').click()

In [13]:
# 혹시 iframe인가? 찾아보니 그렇다

src = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930'

In [14]:
driver = webdriver.Chrome()
driver.get(src)
driver.find_element_by_id('cns_Tab22').click()
html = driver.page_source

C:\Users\21ckw\AppData\Local\Temp/ipykernel_7908/1232055759.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('cns_Tab22').click()


In [15]:
## 경로 계속 바뀐다

In [16]:
soup = BeautifulSoup(html, 'html.parser')
# tr 태그까지
# #bG05RlB6cn > table:nth-child(2) > tbody > tr:nth-child(1)
path = '#bG05RlB6cn > table:nth-child(2) > tbody > tr '
trs = soup.select(path)
trs

[]

In [17]:
len(trs[0])

IndexError: list index out of range

In [18]:
# 라벨값들 출력
# #bG05RlB6cn > table:nth-child(2) > tbody > tr:nth-child(1) > th

path0 = 'tr > th'
trs[0].select(path0)[0].text

IndexError: list index out of range

In [19]:
#매출액
# #RVArcVR1a2 > table:nth-child(2) > tbody > tr:nth-child(1) > td:nth-child(2)
path1 = 'tr> td'

sales = trs[0].select(path1)[0].text
sales

IndexError: list index out of range

In [81]:
# 왜 열이 나왔지? 오히려 좋긴한데..

for tr in trs:
#     print(tr)
    labels = tr.select(path0)[0].text
    sales = tr.select(path1)[0].text
    print(labels, sales)

매출액 669,642
영업이익 123,532
영업이익(발표기준) 123,532
세전계속사업이익 128,441
당기순이익 93,607
  당기순이익(지배) 92,668
  당기순이익(비지배) 939
자산총계 3,757,887
부채총계 996,526
자본총계 2,761,362
  자본총계(지배) 2,679,421
  자본총계(비지배) 81,940
자본금 8,975
영업활동현금흐름 141,444
투자활동현금흐름 -239,889
재무활동현금흐름 4,994
CAPEX 85,372
FCF 56,072
이자발생부채 196,265
영업이익률 18.45
순이익률 13.98
ROE(%) 9.51
ROA(%) 6.87
부채비율 36.09
자본유보율 30,242.29
EPS(원) 1,364
PER(배) 15.89
BPS(원) 39,446
PBR(배) 1.48
현금DPS(원) 354
현금배당수익률 
현금배당성향(%) 25.95
발행주식수(보통주) 5,969,782,550


In [20]:
# 모든 항목을 따로 출력하기 너무 힘듬: read_html 사용 (iframe의 src 사용안함 셀레늄url 사용)
url ='https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930'

driver = webdriver.Chrome()
driver.get(url)
driver.find_element_by_id('cns_Tab22').click()
# time.sleep(1)
html = driver.page_source

df =pd.read_html(html, encoding='utf8')
df

C:\Users\21ckw\AppData\Local\Temp/ipykernel_7908/2187550682.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('cns_Tab22').click()


[                                                   0
 0  삼성전자  005930  SamsungElec  KOSPI : 전기전자  WICS ...
 1  EPS 5,159  BPS 42,447  PER 14.25  업종PER 12.70 ...,
                    0                                1
 0        주가/전일대비/수익률        73,500원 /  +500원 / +0.68%
 1      52Weeks 최고/최저               86,200원 /  68,300원
 2                액면가                             100원
 3           거래량/거래대금           11,736,700주 /  8,637억원
 4               시가총액                      4,387,790억원
 5              52주베타                             0.86
 6         발행주식수/유동비율          5,969,782,550주 / 74.71%
 7             외국인지분율                           52.11%
 8  수익률 (1M/3M/6M/1Y)  -6.13%/ +4.11%/ -9.82%/ -11.45%,
    신용등급  BOND  CP
 0   KIS   NaN NaN
 1    KR   NaN NaN
 2  NICE   NaN NaN
 3   NaN   NaN NaN,
                   0
 0  [기준: 2022.02.08],
                                                 주요주주     보유주식수(보통)  보유지분(%)
 0                          삼성생명보험 외 16인 삼성생명보험 외 16인  1.261675e+09   

In [21]:
len(df)

18

In [22]:
df_fs= df[12]
df_fs.head(5)

주요재무정보                분기                                      \
       주요재무정보 2020/09  (IFRS연결) 2020/12  (IFRS연결) 2021/03  (IFRS연결)   
0         매출액          669642.0          615515.0          653885.0   
1        영업이익          123532.0           90470.0           93829.0   
2  영업이익(발표기준)          123532.0           90470.0           93829.0   
3    세전계속사업이익          128441.0           89745.0           97506.0   
4       당기순이익           93607.0           66071.0           71417.0   

                                                            \
  2021/06  (IFRS연결) 2021/09  (IFRS연결) 2021/12(E)  (IFRS연결)   
0          636716.0          739792.0             752699.0   
1          125667.0          158175.0             150702.0   
2          125667.0          158175.0                  NaN   
3          128824.0          163559.0             153960.0   
4           96345.0          122933.0             113345.0   

                                             
  2022/03(E)  (IFRS연결) 2022/06(E)  (IFRS연결)  
0             742150.0             737297.0  
1             130008.0             132477.0  
2                  NaN                  NaN  
3             132142.0             134560.0  
4              95216.0              97255.0

In [23]:
# 전처리 과정
# 행 열 바꾸기
df_fs = df_fs.T
df_fs.head(3)

0         1           2         3        4   \
주요재무정보 주요재무정보                  매출액      영업이익  영업이익(발표기준)  세전계속사업이익    당기순이익   
분기     2020/09  (IFRS연결)  669642.0  123532.0    123532.0  128441.0  93607.0   
       2020/12  (IFRS연결)  615515.0   90470.0     90470.0   89745.0  66071.0   

                                 5           6          7          8   \
주요재무정보 주요재무정보             당기순이익(지배)  당기순이익(비지배)       자산총계       부채총계   
분기     2020/09  (IFRS연결)    92668.0       939.0  3757887.0   996526.0   
       2020/12  (IFRS연결)    64455.0      1617.0  3782357.0  1022877.0   

                                 9   ...     23        24      25      26  \
주요재무정보 주요재무정보                  자본총계  ...   부채비율     자본유보율  EPS(원)  PER(배)   
분기     2020/09  (IFRS연결)  2761362.0  ...  36.09  30242.29  1364.0   15.89   
       2020/12  (IFRS연결)  2759480.0  ...  37.07  30692.79   949.0   21.09   

                               27      28        29       30         31  \
주요재무정보 주요재무정보              BPS(원)  PBR(배)  현금DPS(원)  현금배당수익률  현금배당성향(%)   
분기     2020/09  (IFRS연결)  39446.0    1.48     354.0      NaN      25.95   
       2020/12  (IFRS연결)  39406.0    2.06    1932.0      NaN     203.62   

                                    32  
주요재무정보 주요재무정보               발행주식수(보통주)  
분기     2020/09  (IFRS연결)  5969782550.0  
       2020/12  (IFRS연결)  5969782550.0  

[3 rows x 33 columns]

In [24]:
# 열이름 연도/월 로 바꾸기
# df_fs.columns # 바로 split 가능? 'MultiIndex' object has no attribute 'split'


In [25]:
# 첫행을 column이름으로 만들고 첫행 삭제
df_fs.columns = df_fs.iloc[0]
df_fs.drop(index='주요재무정보' , axis=0, inplace=True)
# df_fs.drop(index=0 , axis=0) 불가능
df_fs

C:\Users\21ckw\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


('주요재무정보', '주요재무정보')          매출액      영업이익 영업이익(발표기준)  세전계속사업이익     당기순이익  \
분기 2020/09  (IFRS연결)     669642.0  123532.0   123532.0  128441.0   93607.0   
   2020/12  (IFRS연결)     615515.0   90470.0    90470.0   89745.0   66071.0   
   2021/03  (IFRS연결)     653885.0   93829.0    93829.0   97506.0   71417.0   
   2021/06  (IFRS연결)     636716.0  125667.0   125667.0  128824.0   96345.0   
   2021/09  (IFRS연결)     739792.0  158175.0   158175.0  163559.0  122933.0   
   2021/12(E)  (IFRS연결)  752699.0  150702.0        NaN  153960.0  113345.0   
   2022/03(E)  (IFRS연결)  742150.0  130008.0        NaN  132142.0   95216.0   
   2022/06(E)  (IFRS연결)  737297.0  132477.0        NaN  134560.0   97255.0   

('주요재무정보', '주요재무정보')    당기순이익(지배) 당기순이익(비지배)       자산총계       부채총계       자본총계  \
분기 2020/09  (IFRS연결)      92668.0      939.0  3757887.0   996526.0  2761362.0   
   2020/12  (IFRS연결)      64455.0     1617.0  3782357.0  1022877.0  2759480.0   
   2021/03  (IFRS연결)      70928.0      489.0  3928263.0  1185577.0  2742686.0   
   2021/06  (IFRS연결)      94507.0     1838.0  3847777.0  1024534.0  2823243.0   
   2021/09  (IFRS연결)     120572.0     2361.0  4104207.0  1136546.0  2967661.0   
   2021/12(E)  (IFRS연결)  111213.0        NaN  4159861.0  1134332.0  3025529.0   
   2022/03(E)  (IFRS연결)   96633.0        NaN        NaN        NaN        NaN   
   2022/06(E)  (IFRS연결)   93176.0        NaN        NaN        NaN        NaN   

('주요재무정보', '주요재무정보')     ...   부채비율     자본유보율  EPS(원) PER(배)   BPS(원) PBR(배)  \
분기 2020/09  (IFRS연결)     ...  36.09  30242.29  1364.0  15.89  39446.0   1.48   
   2020/12  (IFRS연결)     ...  37.07  30692.79   949.0  21.09  39406.0   2.06   
   2021/03  (IFRS연결)     ...  43.23  30135.47  1044.0  19.54  39126.0   2.08   
   2021/06  (IFRS연결)     ...  36.29  31140.36  1391.0  16.99  40361.0    2.0   
   2021/09  (IFRS연결)     ...   38.3  32225.78  1775.0  14.36  42447.0   1.75   
   2021/12(E)  (IFRS연결)  ...  37.49       NaN  1637.0    NaN  43243.0   1.81   
   2022/03(E)  (IFRS연결)  ...    NaN       NaN  1423.0    NaN      NaN    NaN   
   2022/06(E)  (IFRS연결)  ...    NaN       NaN  1372.0    NaN      NaN    NaN   

('주요재무정보', '주요재무정보')    현금DPS(원) 현금배당수익률 현금배당성향(%)    발행주식수(보통주)  
분기 2020/09  (IFRS연결)       354.0     NaN     25.95  5969782550.0  
   2020/12  (IFRS연결)      1932.0     NaN    203.62  5969782550.0  
   2021/03  (IFRS연결)       361.0     NaN     34.57  5969782550.0  
   2021/06  (IFRS연결)       361.0     NaN     25.95  5969782550.0  
   2021/09  (IFRS연결)       361.0     NaN     20.34  5969782550.0  
   2021/12(E)  (IFRS연결)      NaN     NaN       NaN           NaN  
   2022/03(E)  (IFRS연결)      NaN     NaN       NaN           NaN  
   2022/06(E)  (IFRS연결)      NaN     NaN       NaN           NaN  

[8 rows x 33 columns]

In [26]:
# 날짜만 남기기

df_fs.index
df_fs.index.values

# reset_index() 사용시 전에 있던 인덱스를 반환해줌
df_fs.reset_index(inplace=True)

# df_fs

In [27]:
# .str.split 같은 형식은 헷갈려서 못쓰겠다
# df_fs['level_1'].str.split(' ')[0]

In [28]:
df_fs['level_1'].apply(lambda x : x.split(' ')[0])

0       2020/09
1       2020/12
2       2021/03
3       2021/06
4       2021/09
5    2021/12(E)
6    2022/03(E)
7    2022/06(E)
Name: level_1, dtype: object

In [29]:
df_fs.index = df_fs['level_1'].apply(lambda x : x.split(' ')[0])
df_fs.head()

"(주요재무정보, 주요재무정보)",level_0,level_1,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
level_1,,,,,,,,,,,,,,,,,,,,,
2020/09,분기,2020/09 (IFRS연결),669642.0,123532.0,123532.0,128441.0,93607.0,92668.0,939.0,3757887.0,...,36.09,30242.29,1364.0,15.89,39446.0,1.48,354.0,NaN,25.95,5969782550.0
2020/12,분기,2020/12 (IFRS연결),615515.0,90470.0,90470.0,89745.0,66071.0,64455.0,1617.0,3782357.0,...,37.07,30692.79,949.0,21.09,39406.0,2.06,1932.0,NaN,203.62,5969782550.0
2021/03,분기,2021/03 (IFRS연결),653885.0,93829.0,93829.0,97506.0,71417.0,70928.0,489.0,3928263.0,...,43.23,30135.47,1044.0,19.54,39126.0,2.08,361.0,NaN,34.57,5969782550.0
2021/06,분기,2021/06 (IFRS연결),636716.0,125667.0,125667.0,128824.0,96345.0,94507.0,1838.0,3847777.0,...,36.29,31140.36,1391.0,16.99,40361.0,2.0,361.0,NaN,25.95,5969782550.0
2021/09,분기,2021/09 (IFRS연결),739792.0,158175.0,158175.0,163559.0,122933.0,120572.0,2361.0,4104207.0,...,38.3,32225.78,1775.0,14.36,42447.0,1.75,361.0,NaN,20.34,5969782550.0


In [30]:
# level_0, level_1 열 지우기
df_fs =df_fs.drop(columns =['level_0','level_1'])
df_fs

"(주요재무정보, 주요재무정보)",매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
level_1,,,,,,,,,,,,,,,,,,,,,
2020/09,669642.0,123532.0,123532.0,128441.0,93607.0,92668.0,939.0,3757887.0,996526.0,2761362.0,...,36.09,30242.29,1364.0,15.89,39446.0,1.48,354.0,NaN,25.95,5969782550.0
2020/12,615515.0,90470.0,90470.0,89745.0,66071.0,64455.0,1617.0,3782357.0,1022877.0,2759480.0,...,37.07,30692.79,949.0,21.09,39406.0,2.06,1932.0,NaN,203.62,5969782550.0
2021/03,653885.0,93829.0,93829.0,97506.0,71417.0,70928.0,489.0,3928263.0,1185577.0,2742686.0,...,43.23,30135.47,1044.0,19.54,39126.0,2.08,361.0,NaN,34.57,5969782550.0
2021/06,636716.0,125667.0,125667.0,128824.0,96345.0,94507.0,1838.0,3847777.0,1024534.0,2823243.0,...,36.29,31140.36,1391.0,16.99,40361.0,2.0,361.0,NaN,25.95,5969782550.0
2021/09,739792.0,158175.0,158175.0,163559.0,122933.0,120572.0,2361.0,4104207.0,1136546.0,2967661.0,...,38.3,32225.78,1775.0,14.36,42447.0,1.75,361.0,NaN,20.34,5969782550.0
2021/12(E),752699.0,150702.0,NaN,153960.0,113345.0,111213.0,NaN,4159861.0,1134332.0,3025529.0,...,37.49,NaN,1637.0,NaN,43243.0,1.81,NaN,NaN,NaN,NaN
2022/03(E),742150.0,130008.0,NaN,132142.0,95216.0,96633.0,NaN,NaN,NaN,NaN,...,NaN,NaN,1423.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022/06(E),737297.0,132477.0,NaN,134560.0,97255.0,93176.0,NaN,NaN,NaN,NaN,...,NaN,NaN,1372.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_fs.columns

Index(['매출액', '영업이익', '영업이익(발표기준)', '세전계속사업이익', '당기순이익', '당기순이익(지배)',
       '당기순이익(비지배)', '자산총계', '부채총계', '자본총계', '자본총계(지배)', '자본총계(비지배)', '자본금',
       '영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름', 'CAPEX', 'FCF', '이자발생부채', '영업이익률',
       '순이익률', 'ROE(%)', 'ROA(%)', '부채비율', '자본유보율', 'EPS(원)', 'PER(배)',
       'BPS(원)', 'PBR(배)', '현금DPS(원)', '현금배당수익률', '현금배당성향(%)', '발행주식수(보통주)'],
      dtype='object', name=('주요재무정보', '주요재무정보'))

In [32]:
# csv파일로 저장 후 db에 넣기

In [36]:
# 종목 코드 가져오기
df_codes = pd.read_csv('./Pandas_data_file/universe_code.csv')
df_codes.head(3)

,종목명,현재가,대비,대비(%),거래량,L(일봉)H,종목코드,거래량 (G),거래대금(만원) (H),매출액 영업이익률(%) (L),결산기준 (L),매출액 순이익률(%) (M),결산기준 (M),매출액(백만원) (A1),결산기준 (A1),매출액(백만원) (B1),결산기준 (B1),매출액(백만원) (C1),결산기준 (C1)
0,DL,"78,300","-2,000",-2.49,"341,200",-2.49,A000210,"341,200","2,665,082",11.65,Dec-19,7.32,Dec-19,"9,700,078",Dec-19,"10,984,485",Dec-18,"12,335,536",Dec-17
1,유유제약,"12,400",-250,-1.98,"33,195",-1.98,A000220,"33,195","41,476",8.89,Dec-19,6.12,Dec-19,"90,877",Dec-19,"83,096",Dec-18,"62,852",Dec-17
2,한국앤컴퍼니,"17,000",-300,-1.73,"350,644",-1.73,A000240,"350,644","590,353",20.16,Dec-19,19.26,Dec-19,"847,609",Dec-19,"847,957",Dec-18,"824,846",Dec-17


In [38]:
codes = df_codes['종목코드']

In [40]:
# csv 파일 저장

for code in codes[:10]:
    
    url ='https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}'.format(code[1:])

    driver = webdriver.Chrome()
    driver.get(url)
    driver.find_element_by_id('cns_Tab22').click()
    time.sleep(0.5)
    html = driver.page_source
    
    df = pd.read_html(html, encoding='utf8')
    
    
    
    df.to_csv('./fin/fin_{}'.format(code[1:]))

C:\Users\21ckw\AppData\Local\Temp/ipykernel_7908/4181259907.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('cns_Tab22').click()


KeyboardInterrupt: 